In [1]:
#导入相关包
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selec


tion  import train_test_split
from sklearn import linear_model
from sklearn.metrics import f1_score
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

In [36]:
#读取相关文件
train_path = '../data/train.csv'
test_path = '../data/test.csv'
submit_path = '../data/车辆贷款违约预测挑战赛sample_submit.csv'
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
submit_data = pd.read_csv(submit_path)

In [37]:
inf_count = np.isinf(train_data['outstanding_disburse_ratio']).value_counts()[1]
notna_max = np.sort(train_data['outstanding_disburse_ratio'].values)[-(inf_count+1)]+1
print(notna_max)
#train_data['outstanding_disburse_ratio'][np.isinf(train_data['outstanding_disburse_ratio'])] = np.nan
train_data[np.isinf(train_data)] = notna_max

5000002.0


In [38]:
inf_count = np.isinf(test_data['outstanding_disburse_ratio']).value_counts()[1]
notna_max = np.sort(test_data['outstanding_disburse_ratio'].values)[-(inf_count+1)]+1
print(notna_max)
#train_data['outstanding_disburse_ratio'][np.isinf(train_data['outstanding_disburse_ratio'])] = np.nan
test_data[np.isinf(test_data)] = notna_max

2500002.0


In [40]:
#生成提交文件
submit_data['customer_id'] = test_data['customer_id']
submit_data['loan_default'] = 0

In [41]:
#删除特征
train_data = train_data.drop(['year_of_birth','customer_id','mobileno_flag', 'idcard_flag', 'disbursed_date'],axis=1)
test_data = test_data.drop(['year_of_birth','customer_id','mobileno_flag', 'idcard_flag', 'disbursed_date'],axis=1)

In [42]:
#划分数据集
labels = train_data['loan_default'].values
features_scaler = train_data.drop(['loan_default'],axis=1).values
X_train,X_test, y_train, y_test = train_test_split(features_scaler, labels, test_size = 0.2, random_state = 0)

In [43]:
#标准化特征值
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

#标准化特征值(测试数据)
sc = StandardScaler()
sc.fit(test_data)
test_data_std = sc.transform(test_data)

In [44]:
def calculate_f1(model):
    y_pred = model.predict(X_test_std)
    y_true = y_test
    score = f1_score(y_true, y_pred, average='macro')
    return score

In [63]:
#训练决策树模型
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_std, y_train)
print("f1:",calculate_f1(model))

f1: 0.5233497199406105


In [61]:
print(model.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}


In [64]:
result =(model.predict(test_data_std))
print("预测逾期人数：",np.sum(result))
submit_data['loan_default'] = result
submit_data.to_csv('../data/submit/decisiontree.csv',index=False)

预测逾期人数： 8791


In [50]:
# 用GridSearchCV寻找最优参数（字典）
param = {'criterion':['gini'],'max_depth':[10,20,30,50],'min_samples_leaf':[1,2,3,5,10,15],'min_impurity_decrease':[0.01,0.05,0.1,0.2,0.5]}
grid = GridSearchCV(tree.DecisionTreeClassifier(),param_grid=param,cv=6)
grid.fit(X_train_std, y_train)
print('最优分类器:',grid.best_params_,'最优分数:', grid.best_score_)  # 得到最优的参数和分值

最优分类器: {'criterion': 'gini', 'max_depth': 10, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 1} 最优分数: 0.8235166666666668


In [54]:
#训练决策树模型
model = tree.DecisionTreeClassifier(criterion='gini', max_depth= 100, min_impurity_decrease=0.01, min_samples_leaf= 1)
model = model.fit(X_train, y_train)
print("f1:",calculate_f1(model))

f1: 0.4508813354565922


In [57]:
# 用最佳的参数训练
model = tree.DecisionTreeClassifier(criterion='gini', max_depth= 50, min_impurity_decrease=0.1, min_samples_leaf= 2)
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test_std)
print('训练集分数：', model.score(X_train, y_train),'测试集分数',model.score(X_test,y_test))

训练集分数： 0.8235166666666667 测试集分数 0.8211
